# step 4.2 Machine Learning - Model
creating a model and optimizing it.

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import tree,metrics
from sklearn.neighbors import DistanceMetric
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes, svm
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import make_scorer

## spliting to train and test

In [77]:
dataset=pd.read_csv("finale_normalized_df.csv")

In [78]:
def split_to_train_and_test(dataset, label_column, test_ratio, rand_state):
    TRAINING_FEATURES = dataset.columns[dataset.columns != label_column]
    TARGET_FEATURE = label_column
    X=dataset[TRAINING_FEATURES]
    y= dataset[TARGET_FEATURE]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    return X_train, X_test, y_train, y_test

In [79]:
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = dataset
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9159, 48) (2290, 48) (9159,) (2290,)


## first Model 
ניסיון ראשון ביצירת מודל, שימוש בשלושה סוגים עם היפרפרמטרים ברירת מחדל ללא אופטימיזציה.

In [80]:
clf1 = tree.DecisionTreeClassifier()
clf2 = RandomForestClassifier()
clf3 = KNeighborsClassifier(n_neighbors=3)

algs = ["DT","RandForest","KNN"]
clfs = [clf1,clf2,clf3]
for i,clf in enumerate(clfs):
    clf.fit(X_train,y_train)
    print(algs[i])
    print("======")
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    
    print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
    print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
    
    print(metrics.confusion_matrix(y_test,y_pred))

DT
accuracy on train data 0.9585107544491757
accuracy on test data 0.7462882096069869
[[1585   18   14   32   50   65]
 [  15    1    0    3    3    4]
 [  14    2    0    4    4    6]
 [  34    4    2   21   34    9]
 [  58    5    4   22   81   31]
 [  68    3    3   23   47   21]]
RandForest
accuracy on train data 0.9582923899989082
accuracy on test data 0.7812227074235808
[[1666    4    6   16   44   28]
 [  16    0    1    2    4    3]
 [  15    2    1    3    5    4]
 [  31    2    1   15   40   15]
 [  61    4    3   19   89   25]
 [  75    2    2   19   49   18]]
KNN


C:\Users\linoy\Downloads\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\linoy\Downloads\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

accuracy on train data 0.8492193470902937
accuracy on test data 0.7834061135371179
[[1697   10    4   10   25   18]
 [  15    1    1    5    2    2]
 [  21    1    0    4    3    1]
 [  44    3    2   12   32   11]
 [  78    1    5   28   70   19]
 [  90    5    6   14   36   14]]


הגענו לרמת דיוק של כ-0.78 בק"ננ ועם רמת דיוק על סט המבחן של כ 0.84.
בנוסף ביער ובעץ ההחלטה רואים שיש אוברפיטינג ואחוז דיוק נמוך בהרבה בהשווה לזה של סט הבדיקה
נרצה לנסות וליעל את המודלים.

## Second Model
ננסה למצוא את הפרמטרים הטובים ביותר למודלים במטרה ליעל אותם

In [8]:
dataset=pd.read_csv("finale_normalized_df.csv")

In [10]:
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = dataset
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9159, 48) (2290, 48) (9159,) (2290,)


ניסינו לבדוק סוגי מודלים נוספים:

In [11]:
clf1 = tree.DecisionTreeClassifier()
clf2 = RandomForestClassifier(random_state=5)
clf3 = KNeighborsClassifier(n_neighbors=5)
clf4 = GaussianNB()
clf5 = SVC()

algs = ["DT","RandForest","KNN","NaiveByse","SVC"]
clfs = [clf1,clf2,clf3,clf4,clf5]
for i,clf in enumerate(clfs):
    clf.fit(X_train,y_train)
    print(algs[i])
    print("======")
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    
    print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
    print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
    
    print(metrics.confusion_matrix(y_test,y_pred))

DT
accuracy on train data 0.9585107544491757
accuracy on test data 0.7497816593886463
[[1591   19   13   35   47   59]
 [  16    1    0    2    3    4]
 [  15    2    0    3    5    5]
 [  33    4    2   21   35    9]
 [  59    5    5   23   81   28]
 [  69    4    3   20   46   23]]
RandForest
accuracy on train data 0.9582923899989082
accuracy on test data 0.7764192139737991
[[1656    5    4   16   52   31]
 [  15    0    0    4    4    3]
 [  15    1    1    3    6    4]
 [  30    3    0   19   39   13]
 [  63    4    3   19   86   26]
 [  77    1    2   18   51   16]]
KNN


C:\Users\linoy\Downloads\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\linoy\Downloads\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

accuracy on train data 0.8372093023255814
accuracy on test data 0.7903930131004366
[[1696    0    4   11   35   18]
 [  15    0    1    4    4    2]
 [  20    0    0    1    9    0]
 [  45    0    1   16   35    7]
 [  81    1    2   21   82   14]
 [  89    0    1   15   44   16]]
NaiveByse
accuracy on train data 0.34588928922371437
accuracy on test data 0.34323144104803494
[[ 753    1 1003    7    0    0]
 [   3    0   23    0    0    0]
 [   4    0   23    3    0    0]
 [   3    3   94    2    2    0]
 [   6    0  171   15    8    1]
 [  10    0  150    5    0    0]]
SVC
accuracy on train data 0.8042362703351894
accuracy on test data 0.8026200873362446
[[1730    0    0    0   34    0]
 [  19    0    0    0    7    0]
 [  24    0    0    0    6    0]
 [  56    0    0    0   48    0]
 [  93    0    0    0  108    0]
 [  97    0    0    0   68    0]]


ניתן לראות כי:

    1. DT: ניתן לראות רמת דיוק של כ-0.95 על סט הטסט אך רק 0.75 על המבחן, הדבר מראה על אוברפיטינג של המודל, ננסה לסדר את הפרמטרים שלו במטרה לקבל מודל תקין.
    
    2.RandForest: ניתן לראות רמת דיוק של כ-0.95 על סט הטסט אך רק 0.78 על המבחן, הדבר מראה על אוברפיטינג של המודל, ננסה לסדר את הפרמטרים שלו במטרה לקבל מודל תקין.
        
    3.KNN: כ-0.78 על סט המבחןת ננסה לעלות את רמת הדיוק על ידי מציאת הפרמטר הטוב ביותר
        
    4.NaiveByse: רמת דיוק של כ-0.3 גם בסט המבחן וגם בסט הטסט, מודל זה אינו מתאים לנו
        
    5.SVC: רמת דיוק של כ 0.80 

#### KNN

ניסינו לשפר את מודל הקנ"נ ואכן הצלחנו למצוא פרמטרים ששיפרו את רמת הדיוק שלו לכ- 0.80

In [16]:
def find_best_k_for_KNN(X_train, y_train):
    parameters= {'n_neighbors': [3,5,7,9,11,15,21,23,19,25,27,29]}
    knn= KNeighborsClassifier()
    clf= GridSearchCV(knn,parameters, scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))
    clf.fit(X_train, y_train)
    best_K= clf.best_params_['n_neighbors']
    best_f1_val= clf.best_score_
    return best_K, best_f1_val

best_K, best_accuracy_score=find_best_k_for_KNN(X_train,y_train)

In [15]:
print("bestk", best_K)

bestk 29


In [20]:
clf3 = KNeighborsClassifier(n_neighbors=29)
clf3.fit(X_train,y_train)
print("KNN")
print("======")
y_pred = clf3.predict(X_test)
y_pred_train = clf3.predict(X_train)
    
print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
    
print(metrics.confusion_matrix(y_test,y_pred))

KNN


C:\Users\linoy\Downloads\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\linoy\Downloads\anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

accuracy on train data 0.8137351239218256
accuracy on test data 0.8004366812227074
[[1715    0    0    0   36   13]
 [  15    0    0    1    9    1]
 [  21    0    0    0    7    2]
 [  48    0    0    2   45    9]
 [  84    0    0    4   95   18]
 [  90    0    0    3   51   21]]


#### RandomForst

נתמקד בניסיון לשפר את מודל היער על ידי שינוי פרמטרים:

In [124]:
grid_param ={ 
    "n_estimators":[90,100,115,130],
    "max_depth":range(10,20,1),
    "min_samples_split":range(10,20,1),
    "random_state": range(1,5,1)
}

In [125]:
grid_search = GridSearchCV(estimator=clf2 , param_grid = grid_param, cv=5, n_jobs=-1, verbose=3,scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))

In [126]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(10, 20),
                         'min_samples_split': range(10, 20),
                         'n_estimators': [90, 100, 115, 130],
                         'random_state': range(1, 5)},
             scoring=make_scorer(accuracy_score), verbose=3)

In [128]:
grid_search.best_params_

{'max_depth': 15,
 'min_samples_split': 15,
 'n_estimators': 130,
 'random_state': 2}

In [164]:
clf = RandomForestClassifier(n_estimators=130, max_depth=15,min_samples_split=15, random_state=2)
clf.fit(X_train,y_train)
print("RandForest")
print("======")
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
    
print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
    
print(metrics.confusion_matrix(y_test,y_pred))

RandForest
accuracy on train data 0.8401572224041925
accuracy on test data 0.8109170305676856
[[1731    0    0    0   28    5]
 [  17    0    0    0    8    1]
 [  21    0    0    1    6    2]
 [  49    0    0    2   49    4]
 [  79    0    0    2  109   11]
 [  93    0    0    2   55   15]]


ראינו שיפור ברמת דיור המודל על סט המבחן וירידה בדיוק על סט הבדיקהת מכך ניתן ללמוד כי אנו לא נמצאים במצב של אוברפיטינג יותר.
רמת דיוק על סט המבחן הינה 0.81.

#### DecisionTree

ננסה לשפר גם את מודל העץ החלטה:

In [114]:
grid_param ={
    "min_samples_leaf" : range(1,15,1),
    "max_depth":[5,10,15,30],
    "min_samples_split":range(1,15,1),
}

grid_search = GridSearchCV(estimator=clf1 , param_grid = grid_param, cv=5, n_jobs=-1, verbose=3, scoring=make_scorer(metrics.accuracy_score, greater_is_better=True))

In [117]:
grid_search.fit(X_train, y_train)

In [116]:
grid_search.best_params_

{'max_depth': 5, 'min_samples_leaf': 14, 'min_samples_split': 2}

In [21]:
clf1 = DecisionTreeClassifier(max_depth=5,min_samples_split=2,min_samples_leaf= 14 )
clf1.fit(X_train,y_train)
print("DecisionTree")
print("======")
y_pred = clf1.predict(X_test)
y_pred_train = clf1.predict(X_train)
    
print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
    
print(metrics.confusion_matrix(y_test,y_pred))

DecisionTree
accuracy on train data 0.8131892127961567
accuracy on test data 0.8013100436681223
[[1722    0    0    1   41    0]
 [  16    0    0    0   10    0]
 [  21    0    0    0    9    0]
 [  43    0    0    1   60    0]
 [  87    0    0    2  112    0]
 [ 103    0    0    0   62    0]]


#### SVC

In [65]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.771 total time=   6.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.770 total time=   6.8s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.771 total time=   6.6s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.772 total time=   6.8s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.771 total time=   5.7s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.805 total time=   2.8s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.797 total time=   3.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.803 total time=   3.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.810 total time=   2.9s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.805 total time=   3.2s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.805 total time=   3.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

[CV 2/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.777 total time=   7.8s
[CV 3/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.788 total time=   7.6s
[CV 4/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.784 total time=   8.2s
[CV 5/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.781 total time=   8.1s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.779 total time=   4.7s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.773 total time=   4.9s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.782 total time=   4.8s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.774 total time=   5.1s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.780 total time=   4.3s
[CV 1/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.799 total time=   9.1s
[CV 2/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.794 total time=   8.7s
[CV 3/5] END ....C=1000, gamma=0.01, kernel=rbf;, score=0.806 total time=   9.2s
[CV 4/5] END ....C=1000, gam

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [69]:
grid.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [23]:
clf5 = SVC(C= 10, gamma=0.01 ,kernel='rbf')
clf5.fit(X_train,y_train)
print("SVC")
print("======")
y_pred = clf5.predict(X_test)
y_pred_train = clf5.predict(X_train)
    
print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
    
print(metrics.confusion_matrix(y_test,y_pred))

SVC
accuracy on train data 0.8059831859373294
accuracy on test data 0.8030567685589519
[[1726    0    0    0   37    1]
 [  18    0    0    0    8    0]
 [  23    0    0    0    7    0]
 [  56    0    0    0   48    0]
 [  88    0    0    0  113    0]
 [  94    0    0    0   71    0]]


לא נראה שיפור משמעותי (0.001)

### F1_score

The F1 score is a popular performance measure for classification and often preferred over, for example,
accuracy when data is unbalanced, such as when the quantity of examples belonging to one class 
significantly outnumbers those found in the other class.

https://c3.ai/glossary/data-science/f1-score/

ניתן לראות שהדאטה שלנו אכן לא מאוזן עם כמות גדולה משמעותית של דוגמאות מדורגות 0 לכן נכון יותר לבדוק את:

In [76]:
#RandomForest
clf = RandomForestClassifier(n_estimators=130, max_depth=15,min_samples_split=15, random_state=2)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

rf_test=f1_score(y_test, y_pred, pos_label='positive',average='weighted')

rf_train=f1_score(y_train, y_pred_train, pos_label='positive',average='weighted')

In [45]:
#svc
clf5.fit(X_train,y_train)
y_pred = clf5.predict(X_test)
y_pred_train = clf5.predict(X_train)

svc_testsvc_train=f1_score(y_test, y_pred, pos_label='positive',average='weighted')

svc_train=f1_score(y_train, y_pred_train, pos_label='positive',average='weighted')

In [46]:
#knn
clf3.fit(X_train,y_train)
y_pred = clf3.predict(X_test)
y_pred_train = clf3.predict(X_train)

knn_test=f1_score(y_test, y_pred, pos_label='positive',average='weighted')

knn_train=f1_score(y_train, y_pred_train, pos_label='positive',average='weighted')

In [47]:
#dt
clf1.fit(X_train,y_train)
y_pred = clf1.predict(X_test)
y_pred_train = clf1.predict(X_train)

dt_test=f1_score(y_test, y_pred, pos_label='positive',average='weighted')

dt_train=f1_score(y_train, y_pred_train, pos_label='positive',average='weighted')

In [43]:
model_name=["RandomForest", "SVC", "KNN", "DecisionTree"]
f1_score_train=[rf_train,svc_train,knn_train,dt_train]
f1_score_test=[rf_test,svc_test,knn_test,dt_test]

df_score=pd.DataFrame({"model_name":model_name , "f1_score train":f1_score_train ,"f1_score test":f1_score_test })
df_score

,model_name,f1_score train,f1_score test
0,RandomForest,0.803359,0.764208
1,SVC,0.750725,0.750725
2,KNN,0.773206,0.759393
3,DecisionTree,0.758747,0.746879


ניתן לראות כי הציון הכי טוב על גם סט הבדיקה וגם סט המבחן היה במודל היער.

## third Model
ננסה ליעל את המודל על ידי מניפוליזציה של טבלת הנתונים- ננסה להוסיף עמודות היכולות להשפיע על רמת הדיוק וכן ננסה להוריד עמודות בעלות קורליזציה גבוהה אחת לשנייה או בעלות השפעה נמוכה על המודל

ניסינו להוריד את העמודות בעלי ההשפעה הנמוכה ביותר על המודל אך ראינו שבהורדתן אנחנו מורידים מיכולת הדיוק של המודל ולכן בחרנו שלא להוריד אותם.

In [68]:
fDF= pd.DataFrame({"features":clf.feature_names_in_, "weights": clf.feature_importances_ })
fDF.sort_values("weights", ascending=True).head()

,features,weights
2,XXS,0.000000
10,One_size,0.000605
25,activewear,0.001051
40,poplin,0.001143
29,heavyweight,0.002079


In [69]:
tr_df=dataset.copy(deep=True)

In [73]:
temp= dataset["satin"]
tr_df.drop(["satin"], inplace=True, axis=1)

In [82]:
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = tr_df
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
f1_s=f1_score(y_test, y_pred, pos_label='positive',average='weighted')

In [85]:
print("RandomForest")
print("======")
print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))
print("f1_score Score for test: ",f1_s)

print(metrics.confusion_matrix(y_test,y_pred))

RandomForest
accuracy on train data 0.8492193470902937
accuracy on test data 0.7838427947598253
f1_score Score for test:  0.754112778723497
[[1697   10    4   10   25   18]
 [  15    1    1    5    2    2]
 [  21    1    0    4    3    1]
 [  44    3    2   13   32   10]
 [  78    1    5   28   70   19]
 [  90    5    6   14   36   14]]


In [75]:
tr_df["satin"]=temp

ניסינו להוסיף עמודות הסופרות את מספר המידות ומספר הצבעים הקיימים לאותו הפריט, ההיגיון מאוחרי זה הינו שככל שקיימות יותר אופציות מידה\צבע קהל הקונים של אותו הפריט רחב יותר ועל כן גם מספר הלקוחות עם פוטנציל לדירוג הפריט.

In [77]:
total_size_options=[]
for ind in dataset.index:
    count=0
    if dataset['XXS'][ind]==1: count=count+1
    if dataset['XS'][ind]==1: count=count+1
    if dataset['S'][ind]==1: count=count+1
    if dataset['P'][ind]==1: count=count+1
    if dataset['M'][ind]==1: count=count+1
    if dataset['L'][ind]==1: count=count+1
    if dataset['XL'][ind]==1: count=count+1
    if dataset['XXL'][ind]==1: count=count+1
    if dataset['XXXL'][ind]==1: count=count+1
    if dataset['XXXL'][ind]==1: count=count+1
    if dataset['6XL'][ind]==1: count=count+1
    if dataset['5XL'][ind]==1: count=count+1
    if dataset['One_size'][ind]==1: count=count+1
        
    total_size_options.append(count)
    
dataset["total_size_options"]=total_size_options

In [78]:
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = dataset
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9159, 49) (2290, 49) (9159,) (2290,)


In [81]:
clf.fit(X_train,y_train)
print("RandomForest")
print("======")
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))

print(metrics.confusion_matrix(y_test,y_pred))

RandomForest
accuracy on train data 0.8432143247079376
accuracy on test data 0.8034934497816594
[[1722    0    0    0   38    4]
 [  17    0    0    0    7    2]
 [  20    0    0    1    5    4]
 [  48    0    0    3   47    6]
 [  80    0    0    3  102   16]
 [  96    0    0    3   53   13]]


In [82]:
dataset.drop(["total_size_options"], inplace=True, axis=1)

ניתן לראות ירידה בדיוק המודל לכן החלטנו להוריד את העמודה הזו.

In [83]:
total_color_options=[]
for ind in dataset.index:
    count=0
    if dataset['Blue'][ind]==1: count=count+1
    if dataset['Black'][ind]==1: count=count+1
    if dataset['Grey'][ind]==1: count=count+1
    if dataset['White'][ind]==1: count=count+1
    if dataset['Brown'][ind]==1: count=count+1
    if dataset['Orange'][ind]==1: count=count+1
    if dataset['Pink'][ind]==1: count=count+1
    if dataset['Green'][ind]==1: count=count+1
    if dataset['Purple'][ind]==1: count=count+1
    if dataset['Red'][ind]==1: count=count+1
    if dataset['Yellow'][ind]==1: count=count+1
    if dataset['other_colors'][ind]==1: count=count+1
    if dataset['One_size'][ind]==1: count=count+1
        
    total_color_options.append(count)

dataset["total_color_options"]=total_color_options

In [84]:
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = dataset
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9159, 49) (2290, 49) (9159,) (2290,)


In [85]:
clf.fit(X_train,y_train)
print("RandomForest")
print("======")
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))

print(metrics.confusion_matrix(y_test,y_pred))

RandomForest
accuracy on train data 0.8424500491320013
accuracy on test data 0.8065502183406114
[[1724    0    0    0   32    8]
 [  18    0    0    0    7    1]
 [  21    0    0    1    7    1]
 [  50    0    0    2   46    6]
 [  82    0    0    3  108    8]
 [  94    0    0    3   55   13]]


In [86]:
dataset.drop(["total_color_options"], inplace=True, axis=1)

ניתן לראות ירידה בדיוק המודל לכן החלטנו להוריד את העמודה הזו.

ראינו קורלציה גבוהה בין עמודות ה: (אלסטיין) ועמודת ה (פוליאסטר) לכן החלטנו לנסות להוריד אחת מהן ולראות מה ההשפעה על המודל אך גם כאן הייתה ירידה ברמת דיוק המודל

In [87]:
tr_df=dataset.copy(deep=True)
temp= dataset["elastane"]
tr_df.drop(["elastane"], inplace=True, axis=1)

In [88]:
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = tr_df
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
clf.fit(X_train,y_train)
print("RandomForest")
print("======")
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))

print(metrics.confusion_matrix(y_test,y_pred))

RandomForest
accuracy on train data 0.8378643956763839
accuracy on test data 0.8078602620087336
[[1724    0    0    0   35    5]
 [  18    0    0    0    6    2]
 [  21    0    0    1    7    1]
 [  47    0    0    3   50    4]
 [  79    0    0    4  113    5]
 [  97    0    0    1   57   10]]


In [89]:
tr_df["elastane"]=temp

קיימת ירידה ברמת הדיוק של סט הבדיקה יחד עם סט המבחן לכן הורדנו גם את העמודה הזו

מהסתכלות בשלב הויזואליזציה, ראינו קורלציה גבוהה עד כמעט זהה בין עמודות

(XL, XXL) ועמודות (XXXL, XXXXL)

לכן בחרנו להוריד את העמודות

(xl, xxxl)

In [50]:
temp1= dataset["XL"]
temp2= dataset["XXXL"]
tr_df.drop(["XL","XXXL"], inplace=True, axis=1)
test_ratio, rand_state = 0.2, 42
category_col_name = 'ratings'
dataset = tr_df
X_train, X_test, y_train, y_test = split_to_train_and_test(dataset, category_col_name, test_ratio, rand_state)
clf.fit(X_train,y_train)
print("RandomForest")
print("======")
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

print("accuracy on train data",metrics.accuracy_score(y_true=y_train,y_pred=y_pred_train))
print("accuracy on test data",metrics.accuracy_score(y_true=y_test,y_pred=y_pred))

print(metrics.confusion_matrix(y_test,y_pred))

RandomForest
accuracy on train data 0.8401572224041925
accuracy on test data 0.8056768558951966
[[1730    0    0    0   31    3]
 [  18    0    0    0    6    2]
 [  21    0    0    0    6    3]
 [  49    0    0    2   48    5]
 [  82    0    0    2  104   13]
 [  96    0    0    5   55    9]]


In [51]:
tr_df["XL"]=temp1
tr_df["XXXL"]=temp2

ראינו שלאחר הורדת עמודות אלו לא ניראה שינוי בדיוק במודל, אך מהסתכלות בפיוזן מטריקס ראינו שיפור בזיהוי 5 כוכבים וירידה בזיהוי 4 כוכבים ולכן החלטנו להשאיר את הטבלה כמו שהיא.

מסקנות:

    הגענו למודל הכי טוב ברמת דיוק של כ- 81% ומכך אנו מסיקות כי ניתן לחזות את דירוג הפריט ברמת דיוק מסוימת.
    
    ניתן לראות מהכונפיוזין מטריקס שהרבה טעויות נעשו בסיווג פריטים אשר שיכים לקטגוריית האפס דבר אשר הגיוני כייון שפריטים רבים מדורגים 0 ישר בכניסתם לאתר ורק בהמשך עולים בדירוג.
    
    המודל נבדק ללא פריטים המדורגים 1 או 2 מה שגם יכול היה להשפיע על רמת הדיוק שלו.
    
    בנוסף ניתן לראות שחלק גדול מ4 הכוכבים מסווגים כ5 או 3 כוכבים וכך גם בדירוד 5 הכוכבים מה שמראה על בעייה ביכולת הבסיווג
    

קישורים לאתרים בהם נעשה שימוש:
https://medium.com/analytics-vidhya/random-forest-classifier-and-its-hyperparameters-8467bec755f6